In [1]:
#Imports here
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
from glob import glob
import xgboost as xgb
from sklearn.model_selection import train_test_split

In [2]:
#Functions here

def getCancerTypeNum(df):
    cancer_types = []
    #Get cancer types as 0 or 1
    for sample in df.index:
        #Control as 0
        if "CTRL" in sample:
            cancer_types += [0]
        #Cancer as 1
        else:
            cancer_types += [1]
    
    return cancer_types
    

#get methylation data
def getMethylationData(path):
    methylation_files = glob(path)
    cpg_dict = {}  #key: CpG site, value: percentage methylation
    sample_names = []
    cpg_list = []

    for file in methylation_files:
        sample_id = os.path.basename(file)[0:16].replace(".", "") #get SRR name and status identity, filesnames have cancer or healthy status for convenience
        df = pd.read_csv(file, sep=",")

        #assume columns in file: chr-start-end, perc_methy
        df['cpg'] = df['chr-start-end'].astype(str)
        df['methy'] = df['perc_methy']/100

        for _, row in df.iterrows():
            cpg = row['cpg']
            methy = row['methy']
            if pd.isna(methy): 
                continue
            if cpg not in cpg_dict:
                cpg_dict[cpg] = {}
            cpg_dict[cpg][sample_id] = methy
            if cpg not in cpg_list:
                cpg_list += [cpg]
    
        #sample names
        sample_names += [sample_id]

    #initialize dataframe
    df = pd.DataFrame.from_dict(cpg_dict)

    return df, sample_names

#split into train, validation, test
def splitDatasets(data):
    #Splitting into train validation test dfs
    train_df, temp_df = train_test_split(data, test_size=0.4, random_state=42)
    val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)
    #Get labels for machine learning
    train_labels = getCancerTypeNum(train_df)
    val_labels = getCancerTypeNum(val_df)
    test_labels = getCancerTypeNum(test_df)

    return train_df, val_df, test_df, train_labels, val_labels, test_labels

In [3]:
#prepare dataframe
df_CRCvCTRL, CRCCTRLid = getMethylationData("out/CRCvsCTRL/*.csv")
df_CTRLvSTAD, CTRLSTADid = getMethylationData("out/CTRLvsSTAD/*.csv")
thresholdCRC = 0.8 * 36 #36 samples CRCvCTRL
thresholdSTAD = 0.8 * 53 #53 samples STADvCTRL
df_CRCvCTRL_clean = df_CRCvCTRL.dropna(axis=1, thresh=thresholdCRC)
df_CTRLvSTAD_clean = df_CTRLvSTAD.dropna(axis=1, thresh=thresholdSTAD)
df_list = [df_CRCvCTRL_clean, df_CTRLvSTAD_clean]
df_names = ["CRCvsCTRL", "CTRLvsSTAD"]

Biological data analysis

In [ ]:
#list of CpGs not removed
CRCstay = df_CRCvCTRL_clean.columns.to_list()
STADstay = df_CTRLvSTAD_clean.columns.to_list()


#Extract differentially methylated cpgs with q-values, p-values etc. from methylKit
CRCdmCpGs = pd.read_csv("CRCdiffmeth.tsv", sep=" ")
STADdmCpGs = pd.read_csv("STADdiffmeth.tsv", sep=" ")
CRCdmCpGs['meth.diff'] = CRCdmCpGs['meth.diff'].abs()
STADdmCpGs['meth.diff'] = STADdmCpGs['meth.diff'].abs()
topCRC = CRCdmCpGs.sort_values(by=['meth.diff'], ascending=False)
topSTAD = STADdmCpGs.sort_values(by=['meth.diff'], ascending=False)

#Get pos
listCRCdmCpG = []
listSTADdmCpG = []

for pos in zip(topCRC['chr'], topCRC['start'], topCRC['end']):
    listCRCdmCpG += [f"{pos[0]}-{pos[1]}-{pos[2]}"]

for pos in zip(topSTAD['chr'], topSTAD['start'], topSTAD['end']):
    listSTADdmCpG += [f"{pos[0]}-{pos[1]}-{pos[2]}"]

diffmethCRC = topCRC['meth.diff'].to_list()
diffmethSTAD = topSTAD['meth.diff'].to_list()

#Turn into dataframe
df_diffmethCRC = pd.DataFrame(columns=['pos', 'meth.diff'])
df_diffmethCRC['pos'] = listCRCdmCpG
df_diffmethCRC['meth.diff'] = diffmethCRC

df_diffmethSTAD = pd.DataFrame(columns=['pos', 'meth.diff'])
df_diffmethSTAD['pos'] = listSTADdmCpG
df_diffmethSTAD['meth.diff'] = diffmethSTAD

#For selecting top CpGS
def selectTopCRC(num):
    topCRC = df_diffmethCRC.loc[df_diffmethCRC['pos'].isin(CRCstay)].head(num)
    df_topCRCvCTRL = df_CRCvCTRL_clean[topCRC['pos'].to_list()]
    return df_topCRCvCTRL

def selectTopSTAD(num):
    topSTAD = df_diffmethSTAD.loc[df_diffmethSTAD['pos'].isin(STADstay)].head(num)
    df_topCTRLvSTAD = df_CTRLvSTAD_clean[topSTAD['pos'].to_list()]
    return df_topCTRLvSTAD

#imputer name
imp_names = ["mean", "median"]

X_train, X_val, X_test, y_train, y_val, y_test = splitDatasets(df_CRCvCTRL_clean)
model = xgb.XGBClassifier(importance_type='gain')
model.fit(X_train, y_train)

#Get importance
importancesCRC = model.feature_importances_
#Get score
boosterCRC = model.get_booster()
scoresCRC = boosterCRC.get_score(importance_type='gain')
df_CRCvCTRLXG = df_CRCvCTRL_clean[scoresCRC.keys()]

X_train, X_val, X_test, y_train, y_val, y_test = splitDatasets(df_CTRLvSTAD_clean)
model = xgb.XGBClassifier(importance_type='gain')
model.fit(X_train, y_train)

#Get importance
importancesSTAD = model.feature_importances_
#Get score
boosterSTAD = model.get_booster()
scoresSTAD = boosterSTAD.get_score(importance_type='gain')
df_CTRLvSTADXG = df_CTRLvSTAD_clean[scoresSTAD.keys()]

In [73]:
#Get positions of XGBoost feature selected CpGs
#scoresCRC and scoresSTAD keys contain CpG positions
CRCXGlist = [i for i in scoresCRC.keys()]
STADXGlist = [i for i in scoresSTAD.keys()]

CRCXGpos = [i.split("-") for i in CRCXGlist]
STADXGpos = [i.split("-") for i in STADXGlist]

#Add one for actual CpG position
for i in CRCXGpos:
    i[1] = int(i[1]) + 1
    i[2] = int(i[2]) + 1

for i in STADXGpos:
    i[1] = int(i[1]) + 1
    i[2] = int(i[2]) + 1



In [6]:
df_diffmethCRC.loc[df_diffmethCRC['pos'].isin(CRCXGlist)]

pos  meth.diff
37    chr2-87623968-87623969  60.923077
103  chr16-46399653-46399654  40.079229
144   chr4-49659725-49659726  32.633377
172  chr19-27736157-27736158  29.435484
175  chr18-18518225-18518226  29.066608
188  chr16-46413388-46413389  28.107376

In [7]:
df_diffmethSTAD.loc[df_diffmethSTAD['pos'].isin(STADXGlist)]

pos  meth.diff
102  chr10-42398961-42398962  59.398496
131     chr2-3184905-3184906  54.215686
181     chr1-2627473-2627474  47.647059
193   chr7-61794635-61794636  46.543055
212   chr7-61794661-61794662  44.702703
237   chr9-68425669-68425670  43.154762
252  chr17-21904731-21904732  41.818182
268  chr21-11186861-11186862  39.213368
271  chr16-46399653-46399654  38.820169
303   chr9-68414142-68414143  36.384439
318  chr16-46420954-46420955  35.206831
361  chr16-46423833-46423834  32.135765
365   chr2-89850508-89850509  31.789298
378   chr2-92309814-92309815  30.945840
416   chr7-56441296-56441297  28.291018
459   chr8-46854288-46854289  25.909109

In [ ]:
#Reinitialize dmCpGs without absolute values of methylation difference
CRCdmCpGs = pd.read_csv("CRCdiffmeth.tsv", sep=" ")
STADdmCpGs = pd.read_csv("STADdiffmeth.tsv", sep=" ")

#Get rows by filtering by CpG position. -1 position since positions from methylkit are one lower
CRCXG_tmp = CRCdmCpGs.loc[CRCdmCpGs['chr'].isin([i[0] for i in CRCXGpos])]
CRCXG_tmp = CRCXG_tmp.loc[CRCXG_tmp['start'].isin([i[1]-1 for i in CRCXGpos])]
CRCXG_stats = CRCXG_tmp.loc[CRCdmCpGs['end'].isin([i[2]-1 for i in CRCXGpos])]

STADXG_tmp = STADdmCpGs.loc[STADdmCpGs['chr'].isin([i[0] for i in STADXGpos])]
STADXG_tmp = STADXG_tmp.loc[STADXG_tmp['start'].isin([i[1]-1 for i in STADXGpos])]
STADXG_stats = STADXG_tmp.loc[STADdmCpGs['end'].isin([i[2]-1 for i in STADXGpos])]

In [81]:
CRCXG_stats

chr     start       end strand        pvalue        qvalue  meth.diff
881   chr16  46399653  46399654      +  2.423902e-63  5.814672e-60 -40.079229
1035  chr16  46413388  46413389      +  9.843103e-12  2.071273e-10 -28.107376
1532  chr18  18518225  18518226      +  1.096664e-29  1.753850e-27 -29.066608
1577  chr19  27736157  27736158      +  1.612031e-06  1.578401e-05  29.435484
1690   chr2  87623968  87623969      +  6.119899e-06  5.319187e-05  60.923077
2470   chr4  49659725  49659726      +  9.775418e-27  1.302786e-24 -32.633377

In [ ]:
STADXG_stats

chr     start       end strand        pvalue        qvalue  meth.diff
13     chr1   2627473   2627474      +  6.661365e-03  7.880057e-03 -47.647059
329   chr10  42398961  42398962      +  5.556542e-07  1.338285e-06 -59.398496
670   chr16  46399653  46399654      +  2.703846e-72  6.512171e-70 -38.820169
857   chr16  46420954  46420955      +  3.326604e-30  1.178246e-28 -35.206831
886   chr16  46423833  46423834      +  1.968564e-10  8.527439e-10 -32.135765
1004  chr17  21904731  21904732      +  1.836129e-03  2.435182e-03 -41.818182
1351   chr2   3184905   3184906      +  1.992257e-10  8.537939e-10 -54.215686
1371   chr2  89850508  89850509      +  9.723221e-05  1.649171e-04  31.789298
1476   chr2  92309814  92309815      +  9.308251e-13  5.312507e-12 -30.945840
1734  chr21  11186861  11186862      +  1.663634e-68  2.862026e-66 -39.213368
2106   chr7  56441296  56441297      +  5.853632e-05  1.030584e-04 -28.291018
2136   chr7  61794635  61794636      +  2.155728e-07  5.570856e-07 -46.543055
2137   chr7  61794661  61794662      +  2.207307e-05  4.172894e-05 -44.702703
2207   chr8  46854288  46854289      +  3.467356e-08  1.041281e-07 -25.909109
2296   chr9  68414142  68414143      +  2.809452e-07  7.086405e-07 -36.384439
2328   chr9  68425669  68425670      +  2.022110e-04  3.191495e-04 -43.154762

In [8]:
annotCRC = pd.read_csv("CRCannotated.tsv", sep=" ")
annotSTAD = pd.read_csv("STADannotated.tsv", sep=" ")

In [10]:
print(annotCRC['annot.symbol'].unique())

print(annotSTAD['annot.symbol'].unique())

['TTC34' nan 'PARP2' 'RPPH1' 'LOC646214' 'LINC00910' 'FOXK2' 'ROCK1P1'
 'RMND5A' 'LSP1P4' 'SEC23B' 'MIR3648-1' 'BAGE' 'BAGE3' 'SLC35F1'
 'LINC01606' 'REXO1L1P' 'SAT1']
['TTC34' nan 'DUX4L3' 'LINC00273' 'FLJ36000' 'CCDC200' 'ROCK1P1' 'RMND5A'
 'SEC23B' 'MIR3648-1' 'ZNF595' 'ZNF718' 'COL23A1' 'TMEM242' 'PTPRN2'
 'EPPK1' 'EHMT1' 'SAT1']


In [11]:
#Get annottations of XGBoost selected dmCpGs
CRCXGannot_df = pd.DataFrame()
for chr, start, end in CRCXGpos:
    curr_annot = annotCRC.loc[(annotCRC['seqnames'] == chr) & (annotCRC['start'] == start) & (annotCRC['end'] == end), ['seqnames', 'start', 'end', 'annot.strand', 'annot.symbol', 'annot.type', 'meth.diff']]
    CRCXGannot_df = pd.concat([CRCXGannot_df, curr_annot], ignore_index=True)

STADXGannot_df = pd.DataFrame()
for chr, start, end in STADXGpos:
    curr_annot = annotSTAD.loc[(annotSTAD['seqnames'] == chr) & (annotSTAD['start'] == start) & (annotSTAD['end'] == end), ['seqnames', 'start', 'end', 'annot.strand', 'annot.symbol', 'annot.type', 'meth.diff']]
    STADXGannot_df = pd.concat([STADXGannot_df, curr_annot], ignore_index=True)

In [12]:
#Count number of hypermethylated and hypomethylated CpGs
print("Hypermethylated CpGs:", len(CRCdmCpGs[(CRCdmCpGs['meth.diff'] > 0)]))
print("Hypomethylated CpGs:", len(CRCdmCpGs[(CRCdmCpGs['meth.diff'] < 0)]))
print("Hypermethylated CpGs:", len(STADdmCpGs[(STADdmCpGs['meth.diff'] > 0)]))
print("Hypomethylated CpGs:", len(STADdmCpGs[(STADdmCpGs['meth.diff'] < 0)]))

Hypermethylated CpGs: 227
Hypomethylated CpGs: 0
Hypermethylated CpGs: 481
Hypomethylated CpGs: 0


In [13]:
CRCpromoters = annotCRC.loc[annotCRC['annot.type'] == "hg19_genes_promoters"].dropna(axis='rows', how='any')
CRCfirstexons = annotCRC.loc[annotCRC['annot.type'] == "hg19_genes_firstexons"].dropna(axis='rows', how='any')

STADpromoters = annotSTAD.loc[annotSTAD['annot.type'] == "hg19_genes_promoters"].dropna(axis='rows', how='any')
STADfirstexons = annotSTAD.loc[annotSTAD['annot.type'] == "hg19_genes_firstexons"].dropna(axis='rows', how='any')

In [14]:
CRCpromoters

seqnames     start       end  width strand  meth.diff annot.seqnames  \
57     chr14  20811470  20811471      2      + -48.290598          chr14   
58     chr14  20811470  20811471      2      + -48.290598          chr14   
59     chr14  20811470  20811471      2      + -48.290598          chr14   
193    chr17  41466509  41466510      2      + -28.888889          chr17   
194    chr17  41466509  41466510      2      + -28.888889          chr17   
195    chr17  41466509  41466510      2      + -28.888889          chr17   
196    chr17  41466509  41466510      2      + -28.888889          chr17   
438     chrX  23801304  23801305      2      + -69.480519           chrX   
450     chrX  23801306  23801307      2      + -69.480519           chrX   
462     chrX  23801311  23801312      2      + -69.480519           chrX   
474     chrX  23801326  23801327      2      + -69.480519           chrX   
486     chrX  23801333  23801334      2      + -69.480519           chrX   
498     chrX  23801336  23801337      2      + -69.480519           chrX   
510     chrX  23801379  23801380      2      + -69.480519           chrX   
522     chrX  23801385  23801386      2      + -69.480519           chrX   
534     chrX  23801392  23801393      2      + -69.480519           chrX   
546     chrX  23801480  23801481      2      + -74.615385           chrX   
556     chrX  23801487  23801488      2      + -75.524476           chrX   
566     chrX  23801501  23801502      2      + -72.727273           chrX   
576     chrX  23801504  23801505      2      + -80.909091           chrX   

     annot.start  annot.end  annot.width annot.strand        annot.id  \
57      20810773   20811772         1000            +  promoter:50731   
58      20810773   20811772         1000            +  promoter:50732   
59      20810773   20811772         1000            +  promoter:50733   
193     41466267   41467266         1000            -  promoter:63617   
194     41466267   41467266         1000            -  promoter:63618   
195     41466267   41467266         1000            -  promoter:63619   
196     41466267   41467266         1000            -  promoter:63620   
438     23800762   23801761         1000            +  promoter:75532   
450     23800762   23801761         1000            +  promoter:75532   
462     23800762   23801761         1000            +  promoter:75532   
474     23800762   23801761         1000            +  promoter:75532   
486     23800762   23801761         1000            +  promoter:75532   
498     23800762   23801761         1000            +  promoter:75532   
510     23800762   23801761         1000            +  promoter:75532   
522     23800762   23801761         1000            +  promoter:75532   
534     23800762   23801761         1000            +  promoter:75532   
546     23800762   23801761         1000            +  promoter:75532   
556     23800762   23801761         1000            +  promoter:75532   
566     23800762   23801761         1000            +  promoter:75532   
576     23800762   23801761         1000            +  promoter:75532   

    annot.tx_id  annot.gene_id annot.symbol            annot.type  
57   uc001vxb.1        10038.0        PARP2  hg19_genes_promoters  
58   uc001vxc.3        10038.0        PARP2  hg19_genes_promoters  
59   uc001vxd.3        10038.0        PARP2  hg19_genes_promoters  
193  uc010wib.1    100130581.0    LINC00910  hg19_genes_promoters  
194  uc010czf.3    100130581.0    LINC00910  hg19_genes_promoters  
195  uc010wid.2    100130581.0    LINC00910  hg19_genes_promoters  
196  uc010wie.2    100130581.0    LINC00910  hg19_genes_promoters  
438  uc031tha.1         6303.0         SAT1  hg19_genes_promoters  
450  uc031tha.1         6303.0         SAT1  hg19_genes_promoters  
462  uc031tha.1         6303.0         SAT1  hg19_genes_promoters  
474  uc031tha.1         6303.0         SAT1  hg19_genes_promoters  
486  uc031tha.1         6303.0         SAT1  hg19_genes_p

In [15]:
CRCfirstexons

seqnames     start       end  width strand  meth.diff annot.seqnames  \
61     chr14  20811470  20811471      2      + -48.290598          chr14   
75     chr15  21935517  21935518      2      +  80.000000          chr15   
299    chr21   9826010   9826011      2      + -27.770073          chr21   
403     chr8  86786426  86786427      2      +  58.333333           chr8   
445     chrX  23801304  23801305      2      + -69.480519           chrX   
446     chrX  23801304  23801305      2      + -69.480519           chrX   
447     chrX  23801304  23801305      2      + -69.480519           chrX   
457     chrX  23801306  23801307      2      + -69.480519           chrX   
458     chrX  23801306  23801307      2      + -69.480519           chrX   
459     chrX  23801306  23801307      2      + -69.480519           chrX   
469     chrX  23801311  23801312      2      + -69.480519           chrX   
470     chrX  23801311  23801312      2      + -69.480519           chrX   
471     chrX  23801311  23801312      2      + -69.480519           chrX   
481     chrX  23801326  23801327      2      + -69.480519           chrX   
482     chrX  23801326  23801327      2      + -69.480519           chrX   
483     chrX  23801326  23801327      2      + -69.480519           chrX   
493     chrX  23801333  23801334      2      + -69.480519           chrX   
494     chrX  23801333  23801334      2      + -69.480519           chrX   
495     chrX  23801333  23801334      2      + -69.480519           chrX   
505     chrX  23801336  23801337      2      + -69.480519           chrX   
506     chrX  23801336  23801337      2      + -69.480519           chrX   
507     chrX  23801336  23801337      2      + -69.480519           chrX   
517     chrX  23801379  23801380      2      + -69.480519           chrX   
518     chrX  23801379  23801380      2      + -69.480519           chrX   
519     chrX  23801379  23801380      2      + -69.480519           chrX   
529     chrX  23801385  23801386      2      + -69.480519           chrX   
530     chrX  23801385  23801386      2      + -69.480519           chrX   
531     chrX  23801385  23801386      2      + -69.480519           chrX   
541     chrX  23801392  23801393      2      + -69.480519           chrX   
542     chrX  23801392  23801393      2      + -69.480519           chrX   
543     chrX  23801392  23801393      2      + -69.480519           chrX   
551     chrX  23801480  23801481      2      + -74.615385           chrX   
552     chrX  23801480  23801481      2      + -74.615385           chrX   
553     chrX  23801480  23801481      2      + -74.615385           chrX   
561     chrX  23801487  23801488      2      + -75.524476           chrX   
562     chrX  23801487  23801488      2      + -75.524476           chrX   
563     chrX  23801487  23801488      2      + -75.524476           chrX   
571     chrX  23801501  23801502      2      + -72.727273           chrX   
572     chrX  23801501  23801502      2      + -72.727273           chrX   
573     chrX  23801501  23801502      2      + -72.727273           chrX   
581     chrX  23801504  23801505      2      + -80.909091           chrX   
582     chrX  23801504  23801505      2      + -80.909091           chrX   
583     chrX  23801504  23801505      2      + -80.909091           chrX   

     annot.start  annot.end  annot.width annot.strand         annot.id  \
61      20811230   20811570          341            -  firstexon:52323   
75      21932514   21940739         8226            -  firstexon:55306   
299      9825832    9826011          180            +  firstexon:72516   
403     86786038   86789306         3269            -  firstexon:33704   
445     23801275   23801534          260            +  firstexon:75528   
446     23801275   23801534          260            +  firstexon:75529   
447     23801275   23801534          260            +  firstexon:75530   
457     23801275   23801534          260            +  firstexon:75

In [16]:
STADpromoters

seqnames     start       end  width strand  meth.diff annot.seqnames  \
384     chr17  41381918  41381919      2      + -26.037500          chr17   
692     chr21   9825779   9825780      2      + -47.727273          chr21   
695     chr21   9825786   9825787      2      + -54.949121          chr21   
698     chr21   9825793   9825794      2      + -46.277666          chr21   
701     chr21   9825796   9825797      2      + -42.726095          chr21   
704     chr21   9825798   9825799      2      + -43.520168          chr21   
707     chr21   9825802   9825803      2      + -42.432145          chr21   
1130     chrX  23801304  23801305      2      + -90.909091           chrX   
1142     chrX  23801306  23801307      2      + -90.909091           chrX   
1154     chrX  23801311  23801312      2      + -90.909091           chrX   
1166     chrX  23801326  23801327      2      + -90.909091           chrX   
1178     chrX  23801333  23801334      2      + -90.909091           chrX   
1190     chrX  23801336  23801337      2      + -90.909091           chrX   
1202     chrX  23801379  23801380      2      + -90.909091           chrX   
1214     chrX  23801385  23801386      2      + -90.909091           chrX   
1226     chrX  23801392  23801393      2      + -90.909091           chrX   
1238     chrX  23801480  23801481      2      + -90.000000           chrX   
1248     chrX  23801487  23801488      2      + -90.909091           chrX   
1258     chrX  23801501  23801502      2      + -90.909091           chrX   
1268     chrX  23801504  23801505      2      + -90.909091           chrX   

      annot.start  annot.end  annot.width annot.strand        annot.id  \
384      41381063   41382062         1000            -  promoter:63616   
692       9824832    9825831         1000            +  promoter:72516   
695       9824832    9825831         1000            +  promoter:72516   
698       9824832    9825831         1000            +  promoter:72516   
701       9824832    9825831         1000            +  promoter:72516   
704       9824832    9825831         1000            +  promoter:72516   
707       9824832    9825831         1000            +  promoter:72516   
1130     23800762   23801761         1000            +  promoter:75532   
1142     23800762   23801761         1000            +  promoter:75532   
1154     23800762   23801761         1000            +  promoter:75532   
1166     23800762   23801761         1000            +  promoter:75532   
1178     23800762   23801761         1000            +  promoter:75532   
1190     23800762   23801761         1000            +  promoter:75532   
1202     23800762   23801761         1000            +  promoter:75532   
1214     23800762   23801761         1000            +  promoter:75532   
1226     23800762   23801761         1000            +  promoter:75532   
1238     23800762   23801761         1000            +  promoter:75532   
1248     23800762   23801761         1000            +  promoter:75532   
1258     23800762   23801761         1000            +  promoter:75532   
1268     23800762   23801761         1000            +  promoter:75532   

     annot.tx_id  annot.gene_id annot.symbol            annot.type  
384   uc031ras.1    100874261.0      CCDC200  hg19_genes_promoters  
692   uc021wgv.1    100500862.0    MIR3648-1  hg19_genes_promoters  
695   uc021wgv.1    100500862.0    MIR3648-1  hg19_genes_promoters  
698   uc021wgv.1    100500862.0    MIR3648-1  hg19_genes_promoters  
701   uc021wgv.1    100500862.0    MIR3648-1  hg19_genes_promoters  
704   uc021wgv.1    100500862.0    MIR3648-1  hg19_genes_promoters  
707   uc021wgv.1    100500862.0    MIR3648-1  hg19_genes_promoters  
1130  uc031tha.1         6303.0         SAT1  hg19_genes_promoters  
1142  uc031tha.1         6303.0         SAT1  hg19_genes_promoters  
1154  uc031tha.1         6303.0         SAT1  hg19_genes_promoters  
1166  uc031tha.1         6303.0         SAT1  hg19_genes_promoters  
1178

In [17]:
STADfirstexons

seqnames      start        end  width strand  meth.diff annot.seqnames  \
311     chr17   21904100   21904101      2      + -84.615385          chr17   
314     chr17   21904106   21904107      2      + -84.615385          chr17   
317     chr17   21904130   21904131      2      + -80.000000          chr17   
320     chr17   21904154   21904155      2      + -72.413793          chr17   
323     chr17   21904161   21904162      2      + -72.413793          chr17   
326     chr17   21904173   21904174      2      + -70.000000          chr17   
329     chr17   21904203   21904204      2      + -68.965517          chr17   
332     chr17   21904210   21904211      2      + -66.666667          chr17   
1058     chr8  144940737  144940738      2      + -43.396226           chr8   
1061     chr8  144940741  144940742      2      + -45.098039           chr8   
1064     chr8  144940743  144940744      2      + -44.230769           chr8   
1137     chrX   23801304   23801305      2      + -90.909091           chrX   
1138     chrX   23801304   23801305      2      + -90.909091           chrX   
1139     chrX   23801304   23801305      2      + -90.909091           chrX   
1149     chrX   23801306   23801307      2      + -90.909091           chrX   
1150     chrX   23801306   23801307      2      + -90.909091           chrX   
1151     chrX   23801306   23801307      2      + -90.909091           chrX   
1161     chrX   23801311   23801312      2      + -90.909091           chrX   
1162     chrX   23801311   23801312      2      + -90.909091           chrX   
1163     chrX   23801311   23801312      2      + -90.909091           chrX   
1173     chrX   23801326   23801327      2      + -90.909091           chrX   
1174     chrX   23801326   23801327      2      + -90.909091           chrX   
1175     chrX   23801326   23801327      2      + -90.909091           chrX   
1185     chrX   23801333   23801334      2      + -90.909091           chrX   
1186     chrX   23801333   23801334      2      + -90.909091           chrX   
1187     chrX   23801333   23801334      2      + -90.909091           chrX   
1197     chrX   23801336   23801337      2      + -90.909091           chrX   
1198     chrX   23801336   23801337      2      + -90.909091           chrX   
1199     chrX   23801336   23801337      2      + -90.909091           chrX   
1209     chrX   23801379   23801380      2      + -90.909091           chrX   
1210     chrX   23801379   23801380      2      + -90.909091           chrX   
1211     chrX   23801379   23801380      2      + -90.909091           chrX   
1221     chrX   23801385   23801386      2      + -90.909091           chrX   
1222     chrX   23801385   23801386      2      + -90.909091           chrX   
1223     chrX   23801385   23801386      2      + -90.909091           chrX   
1233     chrX   23801392   23801393      2      + -90.909091           chrX   
1234     chrX   23801392   23801393      2      + -90.909091           chrX   
1235     chrX   23801392   23801393      2      + -90.909091           chrX   
1243     chrX   23801480   23801481      2      + -90.000000           chrX   
1244     chrX   23801480   23801481      2      + -90.000000           chrX   
1245     chrX   23801480   23801481      2      + -90.000000           chrX   
1253     chrX   23801487   23801488      2      + -90.909091           chrX   
1254     chrX   23801487   23801488      2      + -90.909091           chrX   
1255     chrX   23801487   23801488      2      + -90.909091           chrX   
1263     chrX   23801501   23801502      2      + -90.909091           chrX   
1264     chrX   23801501   23801502      2      + -90.909091           chrX   
1265     chrX   23801501   23801502      2      + -90.909091           chrX   
1273     chrX   23801504   23801505      2      + -90.909091           chrX   
1274     chrX   23801504   23801505      2      + -90.909091           chrX   
1275     chrX   23801504   23801505      2      + -90.9